In [ ]:
!pip install --upgrade pip
!pip install -U keras-tuner

In [8]:
import pandas as pd
import numpy as np
df = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/FeaturesExtracted.csv', delimiter =';',engine='python', error_bad_lines=False)

Skipping line 20144: unexpected end of data


In [9]:
list_cn = df.columns.values
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
print ('columns deleted')

nrows = len(df)
for i in range (0,nrows):
    for j in list_cn:
        list_c = df[j].iloc[i]
        list_c = list_c.replace('[','')
        list_c = list_c.replace(']','')
        list1 = np.fromstring (list_c, sep=' ',dtype=float)
        df[j].iat[i] = list1
        df[j].iat[i] = df[j].iloc[i].mean()
    if (i % 10000 == 0):
        print ("Reihe "+ str(i) + " fertig")
print ("Konvertierung abgeschlossen.")

df2 = df.sample(frac=1)
print ('Daten geshuffelt')

df_gesamt = df2.copy()
del df_gesamt['Unnamed: 0']
del df_gesamt['index']
del df_gesamt['path']
del df_gesamt['age']
del df_gesamt['file_wave']
print ('Nicht benötigte Spalten gelöscht')

df_data_x = df_gesamt
del df_gesamt['gender']
print ('gender Spalte gelöscht')

df_gender_y = df2.gender.copy()
print ('DataFrame mit Gender erstellt')

from sklearn.preprocessing import LabelEncoder
# integer encode
label_encoder = LabelEncoder()
df_gender_y = label_encoder.fit_transform(df_gender_y)
print ('Kodierung Gender abgeschlossen. Bereit für Modellerstellung')

columns deleted
Reihe 0 fertig
Reihe 10000 fertig
Reihe 20000 fertig
Konvertierung abgeschlossen.
Daten geshuffelt
Nicht benötigte Spalten gelöscht
gender Spalte gelöscht
DataFrame mit Gender erstellt
Kodierung Gender abgeschlossen. Bereit für Modellerstellung


In [73]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from keras.layers import Dense, Activation

Using TensorFlow backend.


## Definition einer Funktion zum Bau des Modells

In [96]:
def build_model (hp):
    model = keras.Sequential()
    model.add (layers.Dense (34, 
        input_dim = 34, 
        activation = hp.Choice('dense_activation', values = ['relu', 'tanh','sigmoid'], 
        default = 'relu')))
    model.add (layers.Dense (
        units = hp.Int('units', min_value = 34, max_value = 512, step = 32, default = 100),
        activation = 'relu'))
    model.add (layers.Dense (
        units = hp.Int('units', min_value = 34, max_value = 512, step = 32, default = 100), 
        activation = 'relu'))
    model.add (layers.Dense (
        1,
        activation = hp.Choice('dense_activation', values = ['relu', 'tanh','sigmoid'], default = 'sigmoid')))
    model.compile(
        optimizer = keras.optimizers.Adam(
        hp.Choice ('learning_rate',[1e-2,1e-3,1e-4])),
        loss = 'binary_crossentropy',
        metrics = ['binary_crossentropy','accuracy'])
    return model
    #model.add (Dense (34, input_dim = 34, activation = 'relu'))
    #model.add (Dense (100, activation = 'relu'))
    #model.add (Dense (100, activation = 'relu'))
    #model.add (Dense (1, activation = 'sigmoid'))

In [99]:
tuner = RandomSearch(
    build_model,
    objective = 'val_acc',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'Randomsearch',
    project_name = 'units_activation')

In [100]:
tuner.search_space_summary()

In [81]:
# Data split from df_gesamt
# Default: 75% train 25% test data
# Parameters: test_size
# random_state: Reproducing split over multiple function calls
# shuffle: data shuffle before splitting

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split (df2, test_size = 0.2,shuffle = True, random_state = 42)

train_path = '/home/ec2-user/SageMaker/Tensorflow/train.tsv'
test_path = '/home/ec2-user/SageMaker/Tensorflow/test.tsv'

del df_train['Unnamed: 0']
del df_train['index']
del df_train['path']
del df_train['age']
del df_train['file_wave']

del df_test['Unnamed: 0']
del df_test['index']
del df_test['path']
del df_test['age']
del df_test['file_wave']

df_train.to_csv (train_path, sep = '\t', index = False)
df_test.to_csv (test_path, sep = '\t', index = False)

print ('Done!')

Done!


In [82]:
df_train_x = df_train.copy()
del df_train_x['gender']

df_train_y = df_train['gender'].copy()

df_test_x = df_test.copy()
del df_test_x['gender']

df_test_y = df_test['gender'].copy()

In [83]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_train_y = label_encoder.fit_transform(df_train_y)
df_test_y = label_encoder.fit_transform(df_test_y)

In [84]:
np_train_x = df_train_x.to_numpy()

In [85]:
np_test_x = df_test_x.to_numpy()

In [101]:
import time

# Startzeit
t = time.localtime()
current_time = time.strftime("%d.%m.%Y %H:%M:%S", t)
print(current_time, ' HP Search gestartet ...')

tuner.search (np_train_x,df_train_y,epochs = 5, validation_data = (np_test_x,df_test_y))

current_time = time.strftime("%d.%m.%Y %H:%M:%S", t)
print(current_time, ' HP Search beendet')

01.06.2020 12:44:02  HP Search gestartet ...
Train on 16113 samples, validate on 4029 samples
Epoch 1/5
16113/16113 [==============================] - ETA: 3:54 - loss: 4.0277e-04 - binary_crossentropy: 4.0277e-04 - acc: 1.000 - ETA: 21s - loss: 3.3564e-05 - binary_crossentropy: 3.3564e-05 - acc: 1.000 - ETA: 11s - loss: 1.7512e-05 - binary_crossentropy: 1.7512e-05 - acc: 1.00 - ETA: 8s - loss: 1.1846e-05 - binary_crossentropy: 1.1846e-05 - acc: 1.0000 - ETA: 6s - loss: 8.9504e-06 - binary_crossentropy: 8.9504e-06 - acc: 1.000 - ETA: 5s - loss: 7.1923e-06 - binary_crossentropy: 7.1923e-06 - acc: 1.000 - ETA: 5s - loss: 6.1025e-06 - binary_crossentropy: 6.1025e-06 - acc: 1.000 - ETA: 4s - loss: 5.2307e-06 - binary_crossentropy: 5.2307e-06 - acc: 1.000 - ETA: 4s - loss: 4.5769e-06 - binary_crossentropy: 4.5769e-06 - acc: 1.000 - ETA: 3s - loss: 4.0683e-06 - binary_crossentropy: 4.0683e-06 - acc: 1.000 - ETA: 3s - loss: 3.6615e-06 - binary_crossentropy: 3.6615e-06 - acc: 1.000 - ETA: 3s -

Train on 16113 samples, validate on 4029 samples
Epoch 1/5
16113/16113 [==============================] - ETA: 3:34 - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 16s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 9s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.0000 - ETA: 6s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 5s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 4s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 3s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 3s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 2s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 2s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 2s - loss: 0.0000e+00 - binary_crossentropy: 0.0000e+00 - acc: 1.000 - ETA: 2s - loss: 0.0000e+00 - binary_crossentropy: 0.00

Train on 16113 samples, validate on 4029 samples
Epoch 1/5
16113/16113 [==============================] - ETA: 2:57 - loss: 0.5467 - binary_crossentropy: 0.5467 - acc: 1.000 - ETA: 15s - loss: 0.0921 - binary_crossentropy: 0.0921 - acc: 1.000 - ETA: 9s - loss: 0.0500 - binary_crossentropy: 0.0500 - acc: 1.0000 - ETA: 6s - loss: 0.0343 - binary_crossentropy: 0.0343 - acc: 1.000 - ETA: 5s - loss: 0.0261 - binary_crossentropy: 0.0261 - acc: 1.000 - ETA: 4s - loss: 0.0210 - binary_crossentropy: 0.0210 - acc: 1.000 - ETA: 4s - loss: 0.0176 - binary_crossentropy: 0.0176 - acc: 1.000 - ETA: 3s - loss: 0.0154 - binary_crossentropy: 0.0154 - acc: 1.000 - ETA: 3s - loss: 0.0135 - binary_crossentropy: 0.0135 - acc: 1.000 - ETA: 3s - loss: 0.0120 - binary_crossentropy: 0.0120 - acc: 1.000 - ETA: 3s - loss: 0.0107 - binary_crossentropy: 0.0107 - acc: 1.000 - ETA: 2s - loss: 0.0098 - binary_crossentropy: 0.0098 - acc: 1.000 - ETA: 2s - loss: 0.0089 - binary_crossentropy: 0.0089 - acc: 1.000 - ETA: 2

Train on 16113 samples, validate on 4029 samples
Epoch 1/5
16113/16113 [==============================] - ETA: 3:38 - loss: 0.9741 - binary_crossentropy: 0.9741 - acc: 0.0000e+0 - ETA: 18s - loss: 0.0749 - binary_crossentropy: 0.0749 - acc: 0.9231    - ETA: 10s - loss: 0.0390 - binary_crossentropy: 0.0390 - acc: 0.96 - ETA: 7s - loss: 0.0271 - binary_crossentropy: 0.0271 - acc: 0.9722 - ETA: 6s - loss: 0.0207 - binary_crossentropy: 0.0207 - acc: 0.978 - ETA: 5s - loss: 0.0171 - binary_crossentropy: 0.0171 - acc: 0.982 - ETA: 4s - loss: 0.0143 - binary_crossentropy: 0.0143 - acc: 0.985 - ETA: 4s - loss: 0.0123 - binary_crossentropy: 0.0123 - acc: 0.987 - ETA: 3s - loss: 0.0108 - binary_crossentropy: 0.0108 - acc: 0.988 - ETA: 3s - loss: 0.0097 - binary_crossentropy: 0.0097 - acc: 0.990 - ETA: 3s - loss: 0.0089 - binary_crossentropy: 0.0089 - acc: 0.990 - ETA: 3s - loss: 0.0081 - binary_crossentropy: 0.0081 - acc: 0.991 - ETA: 2s - loss: 0.0074 - binary_crossentropy: 0.0074 - acc: 0.992 

Train on 16113 samples, validate on 4029 samples
Epoch 1/5
16113/16113 [==============================] - ETA: 3:14 - loss: 0.7829 - binary_crossentropy: 0.7829 - acc: 0.0000e+0 - ETA: 19s - loss: 0.2067 - binary_crossentropy: 0.2067 - acc: 0.9091    - ETA: 11s - loss: 0.1096 - binary_crossentropy: 0.1096 - acc: 0.95 - ETA: 8s - loss: 0.0743 - binary_crossentropy: 0.0743 - acc: 0.9677 - ETA: 6s - loss: 0.0562 - binary_crossentropy: 0.0562 - acc: 0.975 - ETA: 5s - loss: 0.0443 - binary_crossentropy: 0.0443 - acc: 0.980 - ETA: 4s - loss: 0.0366 - binary_crossentropy: 0.0366 - acc: 0.984 - ETA: 4s - loss: 0.0311 - binary_crossentropy: 0.0311 - acc: 0.986 - ETA: 4s - loss: 0.0274 - binary_crossentropy: 0.0274 - acc: 0.988 - ETA: 3s - loss: 0.0243 - binary_crossentropy: 0.0243 - acc: 0.989 - ETA: 3s - loss: 0.0217 - binary_crossentropy: 0.0217 - acc: 0.990 - ETA: 3s - loss: 0.0197 - binary_crossentropy: 0.0197 - acc: 0.991 - ETA: 2s - loss: 0.0180 - binary_crossentropy: 0.0180 - acc: 0.992 

INFO:tensorflow:Oracle triggered exit
01.06.2020 12:44:02  HP Search beendet


In [102]:
tuner.results_summary()